In [5]:
import numpy as np
from collections import Counter, OrderedDict

In [2]:
with open('coursera_sessions_train.txt') as f:
    file_train = f.readlines()

with open('coursera_sessions_test.txt') as f:
    file_test = f.readlines()
len(file_train), len(file_test)

(50000, 50000)

In [3]:
class OrderedCounter(Counter, OrderedDict):
    def __repr__(self):
        return '%r(%s)' % (self.__class__.__name__ , OrderedDict(self))
    def __reduce__(self):
        return self.__class__, (OrderedDict(self),)

NameError: name 'Counter' is not defined

In [4]:
data_train_v = np.empty(len(file_train), dtype='object')
data_train_p = np.empty(len(file_train), dtype='object')
data_test_v = np.empty(len(file_test), dtype='object')
data_test_p = np.empty(len(file_test), dtype='object')
v_train = {}
p_train = {}
v_test = {}
p_test = {}

def put(d, elem, i):
    if elem in d.keys():
        v = d[elem]
        d[elem] = (v[0] + 1, v[1])
    else:
        d[elem] = (1, i) 

num_p = 0
num_v = 0
for i in xrange(len(file_train)):
    line = file_train[i].strip()
    parts = line.split(';')
    viewed = parts[0].split(',') if parts[0] != '' else []
    purchased = parts[1].split(',') if parts[1] != '' else []
    data_train_v[i] = viewed
    data_train_p[i] = purchased
    for k in viewed:
        if k != '':
            put(v_train, k, num_v)
            num_v += 1
    for k in purchased:
        if k != '':
            put(p_train, k, num_p)
            num_p += 1
        
num_p = 0
num_v = 0
for i in xrange(len(file_test)):
    line = file_test[i].strip()
    parts = line.split(';')    
    viewed = parts[0].split(',') if parts[0] != '' else []
    purchased = parts[1].split(',') if parts[1] != '' else []
    data_test_v[i] = viewed
    data_test_p[i] = purchased
    for k in viewed:
        put(v_test, k, num_v)
        num_v += 1
    for k in purchased:
        put(p_test, k, num_p)
        num_p += 1
        


KeyboardInterrupt: 

In [12]:
data_train_v = np.empty(len(file_train), dtype='object')
data_train_p = np.empty(len(file_train), dtype='object')
data_test_v = np.empty(len(file_test), dtype='object')
data_test_p = np.empty(len(file_test), dtype='object')
v_train = OrderedDict()
p_train = OrderedDict()
v_test = OrderedDict()
p_test = OrderedDict()

for i in xrange(len(file_train)):
    line = file_train[i].strip()
    parts = line.split(';')
    viewed = parts[0].split(',') if parts[0] != '' else []
    purchased = parts[1].split(',') if parts[1] != '' else []
    data_train_v[i] = viewed
    data_train_p[i] = purchased
    for k in viewed:
        if k != '':
            if not k in v_train:
                v_train[k] = 1
            else:
                v_train[k] += 1
    for k in purchased:
        if k != '':
            if not k in p_train:
                p_train[k] = 1
            else:
                p_train[k] += 1
        
    line = file_test[i].strip()
    parts = line.split(';')    
    viewed = parts[0].split(',') if parts[0] != '' else []
    purchased = parts[1].split(',') if parts[1] != '' else []
    data_test_v[i] = viewed
    data_test_p[i] = purchased
    for k in viewed:
        if k != '':
            if not k in v_test:
                v_test[k] = 1
            else:
                v_test[k] += 1
    for k in purchased:
        if k != '':
            if not k in p_test:
                p_test[k] = 1
            else:
                p_test[k] += 1

In [13]:
print p_train['73'], p_train.keys().index('73')

11 573


In [14]:
def make_sort(d):
    return sorted(sorted(d.keys(), key=lambda k: d[k][1]), key=lambda c: d[c][0], reverse=True)

v_train_sorted = make_sort(v_train)
p_train_sorted = make_sort(p_train)

TypeError: 'int' object has no attribute '__getitem__'

In [41]:
def make_sort2(d):
    return sorted(d.keys(), key=lambda c: d[c], reverse=True)

v_train_sorted = make_sort2(v_train)
p_train_sorted = make_sort2(p_train)
v_test_sorted = make_sort2(v_test)
p_test_sorted = make_sort2(p_test)

In [32]:
def get_recommended2(viewed, order, k):
    return sorted(set(viewed), key=lambda s: order.index(s) if s in order else 100000000)[:k]

In [ ]:
p_train.most_common(10)

In [17]:
def out(filename, s):
    f = open(filename, 'w')
    f.write(s)
    f.close()

In [18]:
def get_precision(recommended, purchased, k):
    return len(set(recommended) & set(purchased)) / float(k)

In [19]:
def get_recall(recommended, purchased, k):
    return len(set(recommended) & set(purchased)) / float(len(purchased))

In [ ]:
def comparator_v(x,y):
    x_v = v_train[x] if x in v_train else 0
    y_v = v_train[y] if y in v_train else 0
    if x_v < y_v:
        return -1
    elif x_v > y_v:
        return 1
    else:
        x_v_i = v_train.keys().index(x) if x in v_train else 10000000
        y_v_i = v_train.keys().index(y) if y in v_train else 10000000
        if x_v_i < y_v_i:
            return -1
        else:
            return 1    

def comparator_p(x,y):
    x_p = p_train[x] if x in p_train else 0
    y_p = p_train[y] if y in p_train else 0
    if x_p < y_p:
        return -1
    elif x_p > y_p:
        return 1
    else:
        x_p_i = p_train.keys().index(x) if x in p_train else 10000000
        y_p_i = p_train.keys().index(y) if y in p_train else 10000000
        if x_p_i < y_p_i:
            return -1
        else:
            return 1    

In [49]:
def get_recommended(viewed, stats, k):
    return sorted(set(viewed), key=lambda z: stats[z] if z in stats else 0, reverse=True)[:k]

In [ ]:
recommended = get_recommended(['1','2','3','4','5','6','7','8','9','10'], v_train, 5)
print recommended
totals[0] += get_precision(recommended, 
                           ['2','4','11'], 5)
totals

## Task1

In [26]:
n = 0
totals = [0.0, 0.0, 0.0, 0.0]
for i in xrange(len(data_train_v)):
    if len(data_train_p[i]) > 0:
        n += 1        
        
        n_recommended = 1#min(1, len(set(data_train_v[i])))           
        recommended = get_recommended(data_train_v[i], v_train, n_recommended)
        #recommended = get_recommended2(data_train_v[i], v_train_sorted, n_recommended)
        totals[0] += get_recall(recommended, data_train_p[i], n_recommended)
        totals[1] += get_precision(recommended, data_train_p[i], n_recommended)
        
        n_recommended = 5#min(5, len(set(data_train_v[i])))
        recommended = get_recommended(data_train_v[i], v_train, n_recommended)
        #recommended = get_recommended2(data_train_v[i], v_train_sorted, n_recommended)
        totals[2] += get_recall(recommended, data_train_p[i], n_recommended)
        totals[3] += get_precision(recommended, data_train_p[i], n_recommended)
print totals       
res = ' '.join(['%.2f' % (total / float(n)) for total in totals])
print n
print res
out('task1.txt', res)

[1593.5746142746127, 1842.0, 2973.8075480075486, 766.6000000000226]
3608
0.44 0.51 0.82 0.21


## Task2

In [48]:
n = 0
totals = [0.0, 0.0, 0.0, 0.0]
for i in xrange(len(data_train_v)):
    if len(data_test_p[i]) > 0:
        n += 1        
        
        n_recommended = min(1, len(set(data_test_v[i])))           
        recommended = get_recommended(data_test_v[i], v_train, n_recommended)
        #recommended = get_recommended2(data_test_v[i], v_test_sorted, n_recommended)
        totals[0] += get_recall(recommended, data_test_p[i], 1)
        totals[1] += get_precision(recommended, data_test_p[i], 1)
        
        n_recommended = 5 #min(5, len(set(data_test_v[i])))
        recommended = get_recommended(data_test_v[i], v_train, n_recommended)
        #recommended = get_recommended2(data_test_v[i], v_test_sorted, n_recommended)
        totals[2] += get_recall(recommended, data_test_p[i], 5)
        totals[3] += get_precision(recommended, data_test_p[i], n_recommended)
print totals       
res = ' '.join(['%.2f' % (total / float(n)) for total in totals])
print res
out('task2.txt', res)

KeyboardInterrupt: 

## Task3

In [37]:
n = 0
totals = [0.0, 0.0, 0.0, 0.0]
for i in xrange(len(data_train_p)):
    if len(data_train_p[i]) > 0:
        n += 1        
        
        n_recommended = 1# min(1, len(set(data_train_v[i])))           
        recommended = get_recommended(data_train_v[i], p_train, n_recommended)
        totals[0] += get_recall(recommended, data_train_p[i], n_recommended)
        totals[1] += get_precision(recommended, data_train_p[i], n_recommended)
        
        n_recommended = 5#min(5, len(set(data_train_v[i])))
        recommended = get_recommended(data_train_v[i], p_train, n_recommended)
        totals[2] += get_recall(recommended, data_train_p[i], n_recommended)
        totals[3] += get_precision(recommended, data_train_p[i], n_recommended)
print totals       
res = ' '.join(['%.2f' % (total / float(n)) for total in totals])
print res
out('task3.txt', res)

[2434.349578199578, 2848.0, 3334.595318570318, 908.6000000000362]
0.67 0.79 0.92 0.25


## Task4

In [45]:
n = 0
totals = [0.0, 0.0, 0.0, 0.0]
for i in xrange(len(data_test_p)):
    if len(data_test_p[i]) > 0:
        n += 1        
        
        n_recommended = 1#min(1, len(set(data_test_v[i])))           
        recommended = get_recommended(data_test_v[i], p_train, n_recommended)
        totals[0] += get_recall(recommended, data_test_p[i], n_recommended)
        totals[1] += get_precision(recommended, data_test_p[i], n_recommended)
        
        n_recommended = 5#min(5, len(set(data_test_v[i])))
        recommended = get_recommended(data_test_v[i], p_train, n_recommended)
        totals[2] += get_recall(recommended, data_test_p[i], n_recommended)
        totals[3] += get_precision(recommended, data_test_p[i], n_recommended)
print totals       
res = ' '.join(['%.2f' % (total / float(n)) for total in totals])
print res
out('task4.txt', res)

[1496.4553571428562, 1732.0, 2897.9057938965057, 739.8000000000178]
0.41 0.47 0.79 0.20


In [ ]:
data_test_p[0]